<a href="https://colab.research.google.com/github/mechanic3000/GB_Neural_lang/blob/Lesson_09/IAV_NLP_HW_09.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import tensorflow as tf

import numpy as np
import os
import time

In [2]:
path_to_file = '/content/Понедельник начинается в субботу.TXT'

In [3]:
text = open(path_to_file, 'rb').read().decode(encoding='utf-8')
# length of text is the number of characters in it
print('Length of text: {} characters'.format(len(text)))

Length of text: 439505 characters


In [4]:
print(text[2000:2500])

днем сиденье.
     -- Благодетель!  --  обрадованно произнес горбоносый,  снял с плеча
ружье и сел рядом со мной.
     Бородатый, нерешительно  заглядывая в заднюю дверцу,  сказал:
     -- А можно, я здесь немножко того?..
     Я перегнулся  через  спинку  и помог ему расчистить место,  занятое
спальным мешком и свернутой  палаткой.  Он  деликатно  уселся,  поставив
ружье между коленей.
     -- Дверцу прикройте получше, -- сказал я.
     Все шло как обычно. Машина тронулась. Горбоносый 


In [5]:
text = text + text

In [6]:
# The unique characters in the file
vocab = sorted(set(text))
print('{} unique characters'.format(len(vocab)))

113 unique characters


In [7]:
# Creating a mapping from unique characters to indices
char2idx = {u:i for i, u in enumerate(vocab)}
idx2char = np.array(vocab)

text_as_int = np.array([char2idx[c] for c in text])

In [8]:
text_as_int, text[:500], len(text_as_int), len(text)

(array([50, 97, 91, ...,  0,  1,  0]),
 'Аркадий и Борис Стругацкие. \r\nПонедельник начинается в субботу\r\n\r\n\r\n     Повесть-сказка для научных работников младшего возраста\r\n\r\n                             Но что странное, что непонятнее всего,\r\n                             это то, как авторы могут брать подобные\r\n                             сюжеты, признаюсь, это уж совсем\r\n                             непостижимо, это точно... нет, нет,\r\n                             совсем не понимаю.\r\n                                                  Н',
 879010,
 879010)

### train and target

In [9]:
# The maximum length sentence you want for a single input in characters
seq_length = 100
examples_per_epoch = len(text)//(seq_length+1)

# Create training examples / targets
char_dataset = tf.data.Dataset.from_tensor_slices(text_as_int)

for i in char_dataset.take(5):
    print(idx2char[i.numpy()])

А
р
к
а
д


In [10]:
sequences = char_dataset.batch(seq_length+1, drop_remainder=True)

for item in sequences.take(5):
    print(repr(''.join(idx2char[item.numpy()])))

'Аркадий и Борис Стругацкие. \r\nПонедельник начинается в субботу\r\n\r\n\r\n     Повесть-сказка для научных р'
'аботников младшего возраста\r\n\r\n                             Но что странное, что непонятнее всего,\r\n '
'                            это то, как авторы могут брать подобные\r\n                             сюж'
'еты, признаюсь, это уж совсем\r\n                             непостижимо, это точно... нет, нет,\r\n    '
'                         совсем не понимаю.\r\n                                                  Н.В.Го'


In [11]:
def split_input_target(chunk):
    input_text = chunk[:-1]
    target_text = chunk[1:]
    return input_text, target_text

dataset = sequences.map(split_input_target)

Print the first example input and target values:

In [12]:
for input_example, target_example in  dataset.take(2):
    print('Input data: ', repr(''.join(idx2char[input_example.numpy()])))
    print('Target data:', repr(''.join(idx2char[target_example.numpy()])))

Input data:  'Аркадий и Борис Стругацкие. \r\nПонедельник начинается в субботу\r\n\r\n\r\n     Повесть-сказка для научных '
Target data: 'ркадий и Борис Стругацкие. \r\nПонедельник начинается в субботу\r\n\r\n\r\n     Повесть-сказка для научных р'
Input data:  'аботников младшего возраста\r\n\r\n                             Но что странное, что непонятнее всего,\r\n'
Target data: 'ботников младшего возраста\r\n\r\n                             Но что странное, что непонятнее всего,\r\n '


In [13]:
# Batch size
BATCH_SIZE = 64

# Buffer size to shuffle the dataset
# (TF data is designed to work with possibly infinite sequences,
# so it doesn't attempt to shuffle the entire sequence in memory. Instead,
# it maintains a buffer in which it shuffles elements).
BUFFER_SIZE = 10000

dataset = dataset.shuffle(BUFFER_SIZE).batch(BATCH_SIZE, drop_remainder=True)

dataset

<_BatchDataset element_spec=(TensorSpec(shape=(64, 100), dtype=tf.int64, name=None), TensorSpec(shape=(64, 100), dtype=tf.int64, name=None))>

In [14]:
# Length of the vocabulary in chars
vocab_size = len(vocab)

# The embedding dimension
embedding_dim = 256

# Number of RNN units
rnn_units = 1024

In [15]:
# model = tf.keras.Sequential(
#     [
#         tf.keras.layers.Embedding(vocab_size, embedding_dim),
#         tf.keras.layers.LSTM(rnn_units, return_sequences=True),
#         tf.keras.layers.Dense(vocab_size)
#     ]
# )

In [16]:
# class RNNgenerator(tf.keras.Model):
#     def __init__(self, vocab_size, embedding_dim, batch_size):
#         super(RNNgenerator, self).__init__()
        
#         self.emb = tf.keras.layers.Embedding(vocab_size, embedding_dim)
                                 
#         self.gru1 = tf.keras.layers.GRU(rnn_units,
#                             return_sequences=True,
#                             recurrent_initializer='glorot_uniform')
#         self.gru2 = tf.keras.layers.GRU(rnn_units,
#                             return_sequences=True,
#                             recurrent_initializer='glorot_uniform')
                           
#         self.fc = tf.keras.layers.Dense(vocab_size)

#     def call(self, x):
#         emb_x = self.emb(x)
#         x1 = self.gru1(emb_x)
#         x = x1
#         for _ in range(3):
#             x = self.gru2(x)
#         #x = self.gru1(x)
#         x = (x + x1)/2
#         return self.fc(x)

# model = RNNgenerator(vocab_size, embedding_dim, BATCH_SIZE)

In [17]:
# def build_model(vocab_size, embedding_dim, rnn_units, batch_size):
#     model = tf.keras.Sequential([
#         tf.keras.layers.Embedding(vocab_size, embedding_dim,
#                                   batch_input_shape=[batch_size, None]),
                                 
#         tf.keras.layers.LSTM(rnn_units,
#                             return_sequences=True,
#                             stateful=True,
#                             recurrent_initializer='glorot_uniform'),

#         tf.keras.layers.LSTM(rnn_units,
#                             return_sequences=True,
#                             stateful=True,
#                             recurrent_initializer='glorot_uniform'),

#          tf.keras.layers.LSTM(rnn_units,
#                             return_sequences=True,
#                             stateful=True,
#                             recurrent_initializer='glorot_uniform'),
        
#         tf.keras.layers.LSTM(rnn_units,
#                             return_sequences=True,
#                             stateful=True,
#                             recurrent_initializer='glorot_uniform'),
                                   
#         tf.keras.layers.Dense(vocab_size)
#     ])
#     return model

In [18]:
def build_model(vocab_size, embedding_dim, rnn_units, batch_size):
    model = tf.keras.Sequential([
        tf.keras.layers.Embedding(vocab_size, embedding_dim),
                                 
        tf.keras.layers.GRU(rnn_units,
                            return_sequences=True,
                            stateful=False,
                            recurrent_initializer='glorot_uniform'),

        tf.keras.layers.GRU(rnn_units,
                            return_sequences=True,
                            stateful=False,
                            recurrent_initializer='glorot_uniform'),

         tf.keras.layers.GRU(rnn_units,
                            return_sequences=True,
                            stateful=False,
                            recurrent_initializer='glorot_uniform'),
                                   
        tf.keras.layers.Dense(vocab_size)
    ])
    return model

In [19]:
model = build_model(
    vocab_size=len(vocab),
    embedding_dim=embedding_dim,
    rnn_units=rnn_units,
    batch_size=BATCH_SIZE)

In [20]:
for input_example_batch, target_example_batch in dataset.take(1):
    example_batch_predictions = model(input_example_batch)
    print(example_batch_predictions.shape, "# (batch_size, sequence_length, vocab_size)")

(64, 100, 113) # (batch_size, sequence_length, vocab_size)


In [21]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, None, 256)         28928     
                                                                 
 gru (GRU)                   (None, None, 1024)        3938304   
                                                                 
 gru_1 (GRU)                 (None, None, 1024)        6297600   
                                                                 
 gru_2 (GRU)                 (None, None, 1024)        6297600   
                                                                 
 dense (Dense)               (None, None, 113)         115825    
                                                                 
Total params: 16,678,257
Trainable params: 16,678,257
Non-trainable params: 0
_________________________________________________________________


## Train the model

In [22]:
def loss(labels, logits):
    return tf.keras.losses.sparse_categorical_crossentropy(labels, logits, from_logits=True)

example_batch_loss = loss(target_example_batch, example_batch_predictions)
print("Prediction shape: ", example_batch_predictions.shape, " # (batch_size, sequence_length, vocab_size)")
print("scalar_loss:      ", example_batch_loss.numpy().mean())

Prediction shape:  (64, 100, 113)  # (batch_size, sequence_length, vocab_size)
scalar_loss:       4.7275906


In [23]:
model.compile(optimizer='adam', loss=loss, run_eagerly=True)

### Configure checkpoints

In [24]:
!rm -rf ./training_checkpoints

In [25]:
!ls ./training_checkpoints

ls: cannot access './training_checkpoints': No such file or directory


In [26]:
# Directory where the checkpoints will be saved
checkpoint_dir = './training_checkpoints'
# Name of the checkpoint files
checkpoint_prefix = os.path.join(checkpoint_dir, "ckpt_{epoch}")

checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_prefix,
    save_freq=88*3,
    save_weights_only=True)

### Execute the training

In [27]:
EPOCHS = 20

In [28]:
history = model.fit(dataset, epochs=EPOCHS, callbacks=[checkpoint_callback])

Epoch 1/20


135/135 [==============================] - 37s 243ms/step - loss: 2.7844
Epoch 2/20
135/135 [==============================] - 33s 233ms/step - loss: 2.0137
Epoch 3/20
135/135 [==============================] - 31s 220ms/step - loss: 1.6017
Epoch 4/20
135/135 [==============================] - 32s 225ms/step - loss: 1.3423
Epoch 5/20
135/135 [==============================] - 31s 219ms/step - loss: 1.1342
Epoch 6/20
135/135 [==============================] - 31s 222ms/step - loss: 0.9259
Epoch 7/20
135/135 [==============================] - 31s 221ms/step - loss: 0.7195
Epoch 8/20
135/135 [==============================] - 31s 219ms/step - loss: 0.5317
Epoch 9/20
135/135 [==============================] - 32s 223ms/step - loss: 0.3813
Epoch 10/20
135/135 [==============================] - 31s 219ms/step - loss: 0.2798
Epoch 11/20
135/135 [==============================] - 31s 217ms/step - loss: 0.2106
Epoch 12/20
135/135 [==============================] - 32s 223ms/step - loss: 0.1707


## Generate text

In [35]:
tf.train.latest_checkpoint(checkpoint_dir)

'./training_checkpoints/ckpt_20'

In [36]:
model = build_model(vocab_size, embedding_dim, rnn_units, batch_size=1)
model.load_weights(tf.train.latest_checkpoint(checkpoint_dir))
model.build(tf.TensorShape([1, None]))

In [37]:
model.summary()

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_2 (Embedding)     (None, None, 256)         28928     
                                                                 
 gru_6 (GRU)                 (None, None, 1024)        3938304   
                                                                 
 gru_7 (GRU)                 (None, None, 1024)        6297600   
                                                                 
 gru_8 (GRU)                 (None, None, 1024)        6297600   
                                                                 
 dense_2 (Dense)             (None, None, 113)         115825    
                                                                 
Total params: 16,678,257
Trainable params: 16,678,257
Non-trainable params: 0
_________________________________________________________________


In [52]:
def generate_text(model, start_string):
    # Evaluation step (generating text using the learned model)

    # Number of characters to generate
    num_generate = 500

    # Converting our start string to numbers (vectorizing)
    input_eval = [char2idx[s] for s in start_string]
    input_eval = tf.expand_dims(input_eval, 0)

    # Empty string to store our results
    text_generated = []

    # Low temperature results in more predictable text.
    # Higher temperature results in more surprising text.
    # Experiment to find the best setting.
    temperature = 1

    # Here batch size == 1
    model.reset_states()
    for i in range(num_generate):
        predictions = model(input_eval)
        predictions = tf.squeeze(predictions, 0)
        # using a categorical distribution to predict the character returned by the model
        predictions = predictions / temperature
        predicted_id = tf.random.categorical(predictions, num_samples=1)[-1, 0].numpy()

        # Pass the predicted character as the next input to the model
        # along with the previous hidden state
        input_eval = tf.expand_dims([predicted_id], 0)

        text_generated.append(idx2char[predicted_id])

    return (start_string + ''.join(text_generated))

In [53]:
text_ = generate_text(model, start_string=u"Лес расступился,   мы   переехали   через  мост  и ")
print(text_)

Лес расступился,   мы   переехали   через  мост  и   скух  созажедиехомиленари аму.  пакох злисколо. ологалонил  ги. ву   И ни,
в. Дес?** дия.
даманиви бро и
от. Арай   Хозничегозавось дия  дил    перудржеся нестично     прегамиво   Янаст но
 рный?
   ниль паболерегоце     пнко   испунобакцывамянно вогди  ого   тозалий.  слыйскт Я к
льшимитемичичашет сиянил   ивзажудимно?
оетьктопрнога    Пр.
   скри    ий     сьнол
пабря"Поврицугротомо     я. легрыви  сяк. ожи    причи Онкалорисн.
всмушегр-----и преск  --  Яни   ССл динося идобубъькар


In [34]:
len(text_)

926